In [1]:
from datetime import datetime
from datetime import timedelta

from pymongo import MongoClient
from pandas import DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tushare as ts
from WindPy import w
w.start();


import fdm

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [ ]:
pro = ts.pro_api()

In [2]:
client = MongoClient("192.168.56.1",27017)

In [8]:
pricedb = fdm.CleanData(client).price()
df = pricedb.interface.query(startdate=datetime(
    2017, 12, 1), enddate=datetime(2017, 12, 31))

df

,code,date,open,high,low,close,vwap,adj_factor
0,002909.SZ,2017-12-01,20.90000,21.20000,20.60000,20.92000,20.931431,1.000
1,002051.SZ,2017-12-01,96.41056,96.86272,96.10912,96.31008,96.445706,5.024
2,600847.SH,2017-12-01,29.82924,30.36922,29.82924,30.23888,30.092859,1.862
3,600677.SH,2017-12-01,75.77784,76.21695,73.33137,74.83689,74.551790,6.273
4,600499.SH,2017-12-01,328.41032,329.57904,322.56672,324.02762,325.374449,29.218
5,002228.SZ,2017-12-01,40.47996,40.94206,39.92544,40.66480,40.475072,9.242
6,002084.SZ,2017-12-01,17.52990,17.99901,17.48052,17.80149,17.850974,2.469
7,603289.SH,2017-12-01,20.45000,21.00000,20.17000,20.38000,20.543035,1.000
8,600560.SH,2017-12-01,48.65211,49.28337,48.38157,49.14810,48.849757,4.509
9,002777.SZ,2017-12-01,70.46109,72.58524,70.42063,71.18937,71.406003,2.023


,code,date,open,high,low,close,vwap,adj_factor
0,000001.SZ,2017-12-01,1424.54060,1433.04532,1377.76464,1382.01700,1402.199528,106.309
1,000001.SZ,2017-12-04,1387.33245,1421.35133,1382.01700,1413.90970,1406.206826,106.309
2,000001.SZ,2017-12-05,1397.96335,1434.10841,1392.64790,1413.90970,1419.689623,106.309
3,000001.SZ,2017-12-06,1409.65734,1410.72043,1359.69211,1390.52172,1380.947304,106.309
4,000001.SZ,2017-12-07,1387.33245,1396.90026,1355.43975,1363.94447,1373.389791,106.309
5,000001.SZ,2017-12-08,1370.32301,1391.58481,1355.43975,1391.58481,1377.858117,106.309
6,000001.SZ,2017-12-11,1390.52172,1442.61313,1369.25992,1435.17150,1400.909107,106.309
7,000001.SZ,2017-12-12,1424.54060,1433.04532,1384.14318,1384.14318,1403.924712,106.309
8,000001.SZ,2017-12-13,1382.01700,1403.27880,1369.25992,1395.83717,1384.490325,106.309
9,000001.SZ,2017-12-14,1397.96335,1414.97279,1372.44919,1382.01700,1391.071896,106.309


In [ ]:
tushare = fdm.Tushare(client)
db = tushare.daily_basic()

In [ ]:
tushare.db.client

In [ ]:
MSCI_CN = {'CNY': '302400L.MI',
           'USD': '302400.MI'}
data = w.wsd("302400L.MI", "close", "1999-12-20", "2019-08-26", "",usedf=True)[1]
data.sort_index(inplace = True)
data.to_json('msci_china.bz2')

In [ ]:
data = w.wsd("000300.SH", "close", "1999-12-20", "2019-08-26", "",usedf=True)[1]
data.sort_index(inplace = True)
data.to_json('hs300.bz2')

In [ ]:
index = pd.read_json('msci_china.bz2').sort_index().dropna()
index_ror = index.pct_change()

In [ ]:
index = pd.read_json('hs300.bz2').sort_index().dropna()
index_ror = index.pct_change()

In [ ]:
clean = fdm.CleanData(client)
cdb = clean.pricing()

tdb = fdm.TempDB(client)
actp = tdb['actPrice']

# Build active price vs CSI300
def actprice(code):
    start_date = datetime(2002,1,4)
    close = cdb.query({'code':code,'date':{'$gt':start_date}},['code','date','close'])
    if not close.empty:
        close.set_index(['date'],inplace = True)
        close = close.sort_index().copy()
        close['close'] = close['close'].pct_change()-index_ror['CLOSE']
        close.iat[0,1] = 0
        close['act_price'] = np.cumprod(close['close']+1)
        close.rename(columns={'close':'act_ror'},inplace=True)
        close.reset_index(inplace=True)
        return close
    else:
        return DataFrame

actp.drop()
codes = cdb.stock_codes()
for code in codes:
    df = actprice(code)
    actp.insert_many(df)



In [ ]:
actp.create_indexs(['date','code'])

In [ ]:
tdb = fdm.TempDB(client)
actp = tdb['actPrice']
mratio = tdb['mmRatio']

def maxmin_ratio(ts):
    maxv = np.max(ts)
    minv = np.min(ts)
    pt = ts[-1]
    return (minv-pt)/(minv-maxv)

#codes = actp.distinct('code')
mratio.drop()
for code in codes:
    data = actp.query({'code':code})
    data.set_index('date',inplace=True)
    data.sort_index(inplace=True)

    periods = [3,6,9,12,24,36]
    for n in periods:
        data['mm_ratio_'+str(n)] = data['act_price'].sort_index(ascending = False).rolling(20*n).apply(maxmin_ratio, raw=True)
    
    data.reset_index(inplace=True)
    mratio.insert_many(data)

In [ ]:
mratio.create_indexs(['date','code'])

In [ ]:
data = actp.query({'code':'000001.SZ'})
data.set_index('date',inplace=True)
data.sort_index(inplace=True)

In [ ]:
['CSI','SSE','SZSE','SW']
indexes = pro.index_basic(market='SW')
indexes.head()

# Factor base

In [ ]:
# Size factor

from fdm import Tushare
from fdm import CleanData

# Var
# client = self.get_client()
client = client
startdate = datetime(2019,1,1)
enddate = datetime(2019,2,1)
category = 'mv'
q=0.5

catcode = {'mv': 'total_mv', 'fmv': 'circ_mv'}

# Init database
tdb = Tushare(client).daily_basic()
cdb = CleanData(client).pricing()

# Get category score
catdata = tdb.query({'trade_date': {'$gt': startdate, '$lte': enddate}},
                    projection=['ts_code', 'trade_date', catcode[category]])
catdata = catdata.pivot(
    index='trade_date', columns='ts_code', values=catcode[category])
catdata.sort_index(inplace=True)

# Split stock codes by category score
def split_list_by_quantile(ts):
    low = ts[ts.quantile(q)>ts].index
    high = ts[ts.quantile(1-q)<=ts].index
    return pd.Series({'low':list(low),'high':list(high)})
split_list = catdata.apply(split_list_by_quantile,axis = 1)

# Get return data


In [ ]:
stocklist = split_list.at['2019-01-02','high']
cdb.query({'code':{'$in':stocklist},'date':datetime(2019,1,2)})#{'$gt': startdate, '$lte': enddate}

In [ ]:
DataFrame(cdb.col.col.find({'code':{'$in':stocklist},'date':{'$lte':datetime(2019,1,2)}}).sort([('date',-1)]).limit(2))

In [ ]:
df = cdb.query({'code':'000001.SZ'})
df = df.set_index('date')
df = df.sort_index()

In [ ]:

df['close']['2010-06-01':'2010-09-01'].plot(figsize=(20,5))

In [ ]:
df['close']['2010-06-01':'2010-09-01'].pct_change()

In [ ]:
datasource = fdm.CleanData(client).pricing()
tdb = fdm.TempDB(client)['tdb']
df = datasource.query({'date':{'$gte':datetime(2018,1,1)}})
tdb.insert_many(df)

In [17]:
df = DataFrame(np.arange(25))
df.columns = ['t']

df[(df['t']<10) & (df['t']>2)]

,t
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [136]:
df = pd.read_excel('test.xlsx').set_index('date')
df.reindex(pd.date_range(datetime(2017, 1, 1), datetime(2019, 6, 1),freq='M'))


,000001.SZ,000002.SZ
2017-01-31,NaN,NaN
2017-02-28,000001.SZ,000002.SZ
2017-03-31,000001.SZ,000002.SZ
2017-04-30,NaN,NaN
2017-05-31,000001.SZ,000002.SZ
2017-06-30,000001.SZ,000002.SZ
2017-07-31,000001.SZ,000002.SZ
2017-08-31,000001.SZ,000002.SZ
2017-09-30,NaN,NaN
2017-10-31,000001.SZ,000002.SZ
